In [1]:
import numpy as np
import pickle
import os
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
file_path = './data/motion/smplx/train/Ballet_001_001_00.npy'
data = np.load(file_path, allow_pickle=True).item()
print(type(data))

<class 'dict'>


In [3]:
print(data.keys())

dict_keys(['transl', 'poses', 'betas', 'global_orient', 'meta'])


In [7]:
data['meta']

{'gender': 'female'}

In [8]:
for key in data.keys():
    if key != 'meta':
        print(key, data[key].shape)

transl (2293, 3)
poses (2293, 165)
betas (2293, 16)
global_orient (2293, 3)


In [9]:
folder_path = './data/motion/smplx/train'
train_file_names = (sorted([f for f in os.listdir(folder_path) if f.endswith('.npy')]))

In [11]:
train_file_names[100:105]

['Lumba_007_004_00.npy',
 'Lumba_007_004_01.npy',
 'Lumba_008_001_00.npy',
 'Lumba_008_001_01.npy',
 'Lumba_009_002_00.npy']

In [12]:
file_loc = './train_data_smplx.pkl'
train_data_smplx = np.load(file_loc, allow_pickle=True)

In [14]:
type(train_data_smplx)

list

In [15]:
len(train_data_smplx)

994

In [16]:
type(train_data_smplx[0])

tuple

In [17]:
type(train_data_smplx[0][0])

numpy.ndarray

In [22]:
train_data_smplx[10][1].shape

(300, 187)

In [23]:
test_loc = './test_data_smplx.pkl'
test_data_smplx = np.load(test_loc, allow_pickle=True)

In [24]:
len(test_data_smplx)

253

In [25]:
test_data_smplx[10][1].shape

(300, 187)

In [26]:
train_data = np.load('train_data_smplx.pkl', allow_pickle=True)
test_data = np.load('test_data_smplx.pkl', allow_pickle=True)

In [27]:
all_train_data = np.concatenate([np.concatenate((T, D)) for T, D in train_data])

In [28]:
all_train_data.shape

(596400, 187)

In [29]:
train_mean = all_train_data.mean(axis=0)
train_std = all_train_data.std(axis=0)

In [30]:
train_mean.shape

(187,)

In [31]:
train_std.shape

(187,)

In [32]:
def normalize_data(data, mean, std):
    return (data - mean) / (std+1e-10)

In [33]:
normalized_train_data = [(normalize_data(T, train_mean, train_std), normalize_data(D, train_mean, train_std)) for T, D in train_data]

In [34]:
type(normalized_train_data)

list

In [35]:
normalized_train_data[0][0].shape

(300, 187)

In [36]:
train_mean.shape

(187,)

In [37]:
train_std.shape

(187,)

In [38]:
normalized_test_data = [(normalize_data(T, train_mean, train_std), normalize_data(D, train_mean, train_std)) for T, D in test_data]

In [39]:
type(normalized_test_data)

list

In [40]:
len(normalized_test_data)

253

## The below all are in axis-angle format only (T, 187)

In [41]:
## The below all are in axis-angle format only (T, 187)
np.save('./new_data/normalized_train_data_smplx.npy', normalized_train_data)
np.save('./new_data/normalized_test_data_smplx.npy', normalized_test_data)
np.save('./new_data/train_mean_smplx.npy', train_mean)
np.save('./new_data/train_std_smplx.npy', train_std)

## Now exploring the axis-magnitude format

In [43]:
def convert_to_axis_magnitude(data):
    '''data [T, 187]'''
    poses = data[:, 6:171].reshape(-1, 55, 3)
    magnitude = np.linalg.norm(poses, axis=2, keepdims=True)
    unit_axis = poses / (magnitude + 1e-10)
    axis_magnitude = np.concatenate([unit_axis, magnitude], axis=2).reshape(poses.shape[0], -1)
    new_data = np.concatenate([data[:, :6], axis_magnitude, data[:, 171:]], axis=1)
    return new_data

In [44]:
def convert_to_axis_angle(data):
    '''data [T, 242]'''
    axis_magnitude = data[:, 6:226].reshape(-1, 55, 4)
    axis = axis_magnitude[:, :, :3]
    magnitude = axis_magnitude[:, :, 3:]
    poses = axis * (magnitude+1e-10)
    poses = poses.reshape(axis_magnitude.shape[0], -1)
    new_data = torch.cat([data[:, :6], poses, data[:, 226:]], axis=1)
    return new_data

In [47]:
import torch

In [48]:
a = np.random.rand(300, 187)
b = convert_to_axis_magnitude(a)
c = convert_to_axis_angle(torch.from_numpy(b))

In [55]:
a[0][6:10]

array([0.83486477, 0.1363486 , 0.63186234, 0.69240692])

In [56]:
b[0][6:10]

array([0.79069653, 0.12913513, 0.59843388, 1.0558599 ])

In [57]:
c[0][6:10]

tensor([0.8349, 0.1363, 0.6319, 0.6924], dtype=torch.float64)

In [58]:
def smplx_to_pos3d(data):

    smplx = SMPLX(model_path='./smplx', betas=data[:, 171:187][:, :10], gender='male', \
        batch_size=len(data[:, 171:187]), num_betas=10, use_pca=False, use_face_contour=True, flat_hand_mean=True)
    
    
    keypoints3d = smplx.forward(
        global_orient=torch.from_numpy(data[:, 3:6]).float(),
        body_pose=torch.from_numpy(data[:, 6:171][:, 3:66]).float(),
        jaw_pose=torch.from_numpy(data[:, 6:171][:, 66:69]).float(),
        leye_pose=torch.from_numpy(data[:, 6:171][:, 69:72]).float(),
        reye_pose=torch.from_numpy(data[:, 6:171][:, 72:75]).float(),
        left_hand_pose=torch.from_numpy(data[:, 6:171][:, 75:120]).float(),
        right_hand_pose=torch.from_numpy(data[:, 6:171][:, 120:]).float(),
        transl=torch.from_numpy(data[:, :3]).float(),
        betas=torch.from_numpy(data[:, 171:187][:, :10]).float()
        ).joints.detach().numpy()[:, :55]

    nframes = keypoints3d.shape[0]
    return keypoints3d.reshape(nframes, -1)

In [59]:
def smplx_to_pos3d_torch(data):

    smplx = SMPLX(model_path='./smplx', betas=data[:, 171:187][:, :10], gender='male', \
        batch_size=len(data[:, 171:187]), num_betas=10, use_pca=False, use_face_contour=True, flat_hand_mean=True)
    
    
    keypoints3d = smplx.forward(
        global_orient=(data[:, 3:6]).float(),
        body_pose=(data[:, 6:171][:, 3:66]).float(),
        jaw_pose=(data[:, 6:171][:, 66:69]).float(),
        leye_pose=(data[:, 6:171][:, 69:72]).float(),
        reye_pose=(data[:, 6:171][:, 72:75]).float(),
        left_hand_pose=(data[:, 6:171][:, 75:120]).float(),
        right_hand_pose=(data[:, 6:171][:, 120:]).float(),
        transl=(data[:, :3]).float(),
        betas=(data[:, 171:187][:, :10]).float()
        ).joints[:, :55]

    nframes = keypoints3d.shape[0]
    return keypoints3d.reshape(nframes, -1)

### Potential Problem, you did normalization first and then axis-magnitude conversion, but after this conversion the data is no longer normalized.